In [1]:
### Code for reproducing simulated data experiments found in https://arxiv.org/abs/2208.08401. 

source("DtACI.R")
source("MVP.R")
source("AgACI.R")
library(ggplot2)
library(gridExtra)

In [2]:
### Generate simulated data that alternates between two states with different rates of fluctuation
generateTwoRegime <- function(T,gaps,sigma1,sigma2,alpha=0.1,startState=1){
  mut <- 0
  prevJump <- 0
  curState <- startState
  betas <- rep(0,T)
  errs <- rep(0,T)
  mus <- rep(0,T)
  muForPredicts <- rep(0,T)  ### Optional sequence of predicted means 
  alphaStars <- rep(0,T)
  for(t in 1:T){
    mus[t] <- mut
    #muForPredicts[t] <- mean(mus[max(t-200,1):(t-1)])
    #betas[t] <- pnorm(rnorm(1,mut,1),0,3,lower.tail=FALSE)
    #alphaStars[t] <- pnorm(qnorm(1-alpha,mut),0,3,lower.tail=FALSE)
    betas[t] <- pnorm(rnorm(1,mut),muForPredicts[t],lower.tail=FALSE)
    alphaStars[t] <- pnorm(qnorm(1-alpha,mut),muForPredicts[t],lower.tail=FALSE)
    if(t %% 5 == 0){
        if(curState == 1){
          errs[t] <- rnorm(1,0,sigma1)
          mut <- mut + 0.5*prevJump + 0.5*errs[t]
        }else{
          errs[t] <- rnorm(1,0,sigma2)
          mut <- mut + errs[t]
        }
        prevJump <- mut - mus[t]
    }
    if(t %% gaps == 0){
      curState <- 1-curState
    }
  }
  return(list(betas,mus,muForPredicts,alphaStars))
}

### Returns 1 if x>0, -1 otherwise
sgn <- function(x){
    return(2*(x>0)-1)
}

### Generate simulated data with large jump discontinuities between two states with a large gap between their means
generateJumpSequences <- function(T,bigGap,smallGap,g1,g2,startState=0){
  mut <- (1-startState)*g1/2 + startState*g2/2
    mut <- -mut
  prevJump <- 0
  curState <- startState
  betas <- rep(0,T)
  errs <- rep(0,T)
  mus <- rep(0,T)
  muForPredicts <- rep(0,T)
  alphaStars <- rep(0,T)
  for(t in 1:T){
    mus[t] <- mut
    #muForPredicts[t] <- mean(mus[max(t-200,1):(t-1)])
    #betas[t] <- pnorm(rnorm(1,mut,1),0,3,lower.tail=FALSE)
    #alphaStars[t] <- pnorm(qnorm(1-alpha,mut),0,3,lower.tail=FALSE)
    betas[t] <- pnorm(rnorm(1,mut),0,lower.tail=FALSE)
    alphaStars[t] <- pnorm(qnorm(1-alpha,mut),muForPredicts[t],lower.tail=FALSE)
    if(t %% smallGap == 0){ ### make a smaller fluctuation
      mut <- -mut 
    }
    if(t %% bigGap == 0){ ### Swap between states
      curState <- 1-curState
      mut <- ((1-curState)*g1/2 + curState*g2/2)*(-sgn(mut))
    }
  }
  return(list(betas,mus,muForPredicts,alphaStars))

}

### Generate data with no distribution shift
generateBetasFixedMus <- function(mus,muForPredicts){
  betas <- rep(0,length(mus))
  for(t in 1:length(mus)){
    #betas[t] <- pnorm(rnorm(1,mus[t]),0,3,lower.tail=FALSE)
    betas[t] <- pnorm(rnorm(1,mus[t]),muForPredicts[t],lower.tail=FALSE)
  }
  return(betas)
}

### Run DtACI, AgACI, and MVP
oneExperimentFixedMus <- function(mus,muForPredicts,alpha,gammaGrid,Keps,eta=2.72,sigma=1/1000,alphaInit=alpha){
  betas <- generateBetasFixedMus(mus,muForPredicts)
  
  dtaciRes <- conformalAdaptStable(betas,alpha,gammaGrid,sigma=sigma,eta=eta,alphaInit=alphaInit)
  agaciRes <- agaci(betas,alpha,gammaGrid,alphaInit=alphaInit)
  mvpRes <- MVP(1-betas,alpha,Keps = Keps)
  mvpRes[[1]] <- 1-mvpRes[[1]]
  
  return(list(dtaciRes[[5]],agaciRes[[1]],mvpRes[[1]],dtaciRes[[7]],agaciRes[[3]]))
}

In [3]:
### Generate data
set.seed(1)
alpha=0.1

sigma1 <- sqrt(0.006)
sigma2 <- sqrt(0.006)
T <- 6000
gaps <- 6000
datFluc <- generateTwoRegime(T,gaps,sigma1,sigma2)

sigma1 <- 0
sigma2 <- 0
T <- 6000
gaps <- 6000
datStat <- generateTwoRegime(T,gaps,sigma1,sigma2)

T <- 6000
smallGap <- 500
bigGap <- 2000
g1 <- 0.15
g2 <- 3
datJump <- generateJumpSequences(T,bigGap,smallGap,g1,g2)

mus <- list(datStat[[2]],datFluc[[2]],datJump[[2]])
muForPredicts <- list(datStat[[3]],datFluc[[3]],datJump[[3]])
alphaStars <- list(datStat[[4]],datFluc[[4]],datJump[[4]])
gapsVec <- c(6000,6000,2000)

In [4]:
### Run DtACI, AgACI, and MVP

nTrials <- 100
alpha <- 0.1
gammaGrid <- c(0.001,0.002,0.004,0.008,0.0160,0.032,0.064,0.128)
Keps <- 2.12

diffList <- list()
diffSqList <- list()
diffSdList <- list()
alphaList <- list()
alphaSqList <- list()
alphaSdList <- list()
gammaList <- list()
gammaSqList <- list()
gammaSdList <- list()
covList <- list()
covSqList <- list()
covSdList <- list()
covDiffList <- list()
covDiffSqList <- list()
covDiffSdList <- list()
covCumulativeDiffList <- list()
covCumulativeDiffSqList <- list()
covCumulativeDiffSdList <- list()

allCovs <- list()

for(i in 1:length(mus)){
    T <- length(mus[[i]])
    diffList[[i]] <- matrix(0,nrow=T,ncol=3)
    diffSqList[[i]] <- matrix(0,nrow=T,ncol=3)
    alphaList[[i]] <- matrix(0,nrow=T,ncol=3)
    alphaSqList[[i]] <-  matrix(0,nrow=T,ncol=3)
    gammaList[[i]] <- matrix(0,nrow=T,ncol=2)
    gammaSqList[[i]] <-  matrix(0,nrow=T,ncol=2)
    covList[[i]] <- matrix(0,nrow=T,ncol=3)
    covSqList[[i]] <-  matrix(0,nrow=T,ncol=3)
    covDiffList[[i]] <- matrix(0,nrow=T,ncol=2)
    covDiffSqList[[i]] <-  matrix(0,nrow=T,ncol=2)
    covCumulativeDiffList[[i]] <- matrix(0,nrow=T,ncol=3)
    covCumulativeDiffSqList[[i]] <-  matrix(0,nrow=T,ncol=3)
    
    allCovs[[i]] <- matrix(0,nrow=T,ncol=3)
    for(j in 1:nTrials){
      if(i==1){
         alphaInit <- alpha   
      }else{
         alphaInit <- alphaStars[[i]][[1]]
      }
      oneTrialRes <- oneExperimentFixedMus(mus[[i]],muForPredicts[[i]],alpha,gammaGrid,Keps,alphaInit=alphaInit) 
      oneTrialAlphas <- oneTrialRes[1:3]
      oneTrialGammas <- oneTrialRes[4:5]
      differences <- lapply(oneTrialAlphas,function(x){abs(x-alphaStars[[i]])})
      differences <- matrix(unlist(differences),ncol=3)
      diffList[[i]] <- diffList[[i]] + differences/nTrials
      diffSqList[[i]] <- diffSqList[[i]] + differences^2/nTrials
      alphaList[[i]] <- alphaList[[i]] + matrix(unlist(oneTrialAlphas),ncol=3)/nTrials
      alphaSqList[[i]] <- alphaSqList[[i]] + matrix(unlist(oneTrialAlphas)^2,ncol=3)/nTrials
      gammaList[[i]] <- gammaList[[i]] + matrix(unlist(oneTrialGammas),ncol=2)/nTrials
      gammaSqList[[i]] <- gammaSqList[[i]] + matrix(unlist(oneTrialGammas)^2,ncol=2)/nTrials
      coverages <-  lapply(oneTrialAlphas,function(x){pnorm(qnorm(pmin(pmax(1-x,0),1),muForPredicts[[i]]),mus[[i]]) })
      allCovs[[i]] <- allCovs[[i]] + matrix(unlist(coverages),ncol=3)/nTrials
      coverageDiffs <-  matrix(abs(unlist(coverages) - (1-alpha)),ncol=3)
      covList[[i]] <- covList[[i]] + coverageDiffs/nTrials
      covSqList[[i]] <- covSqList[[i]] + coverageDiffs^2/nTrials
      covDiffList[[i]] <- covDiffList[[i]] + apply(coverageDiffs[,2:3],2,function(x){x - coverageDiffs[,1]})/nTrials
      covDiffSqList[[i]] <-  covDiffSqList[[i]] + apply(coverageDiffs[,2:3],2,function(x){x - coverageDiffs[,1]})^2/nTrials
      cumulativeCoverageDiffs <- t(sapply(1:length(mus[[i]]),function(x){if(x!=1){apply(coverageDiffs[1:x,],2,sum)}else{coverageDiffs[1,]}}))
      covCumulativeDiffList[[i]] <-  covCumulativeDiffList[[i]] + cumulativeCoverageDiffs/nTrials
      covCumulativeDiffSqList[[i]] <- covCumulativeDiffSqList[[i]] + cumulativeCoverageDiffs^2/nTrials
    }
    alphaList[[i]] <- cbind(alphaList[[i]],alphaStars[[i]])
    alphaSqList[[i]] <- cbind(alphaSqList[[i]],alphaStars[[i]]^2)
    diffSdList[[i]] <- sqrt(pmax(diffSqList[[i]] - diffList[[i]]^2,0))/sqrt(nTrials)
    alphaSdList[[i]] <- sqrt(pmax(alphaSqList[[i]] - alphaList[[i]]^2,0))/sqrt(nTrials)
    gammaSdList[[i]] <- sqrt(pmax(gammaSqList[[i]] - gammaList[[i]]^2,0))/sqrt(nTrials)
    covSdList[[i]] <-  sqrt(pmax(covSqList[[i]] - covList[[i]]^2,0))/sqrt(nTrials)
    covDiffSdList[[i]] <- sqrt(pmax(covDiffSqList[[i]] - covDiffList[[i]]^2,0))/sqrt(nTrials)
    covCumulativeDiffSdList[[i]] <- sqrt(pmax(covCumulativeDiffSqList[[i]] - covCumulativeDiffList[[i]]^2,0))/sqrt(nTrials)
}


In [5]:
### Plot results

library(tidyr)
library(latex2exp)
library(cowplot)
library(dplyr)

localMean <- function(seq,index,bandwidth,bottom=300){
  bottomIndex <- max(1,index-bottom+1)
  weights <- (bandwidth)^(bottomIndex:index)
  weights <- weights/sum(weights)
  return(sum(weights*seq[bottomIndex:index]))
}
makePlot <- function(dataMat,sdMat,T=2000,gaps=1000,smoothingWidth=50,cols = c("dtaci","agaci","mvp","alphaStar"),
                     xlabel = "Time",ylabel = "Error",colors=c(4,2,3,1),legendNames=cols,timeCutoff=0){
  T <- nrow(dataMat)
  if(smoothingWidth>0){
    for(i in 1:length(cols)){
      dataMat[,i] <- sapply(1:T,function(x){localMean(dataMat[,i],x,bandwidth=1,bottom=smoothingWidth)})
      sdMat[,i] <- sapply(1:T,function(x){localMean(sdMat[,i],x,bandwidth=1,bottom=smoothingWidth)})
    }
  }

  colnames(dataMat) <- cols
  colnames(sdMat) <- cols
  sdMat <- as.data.frame(sdMat)
  sdMat$Time <-1:T
  dataMat <- as.data.frame(dataMat)
  dataMat$Time <-1:T
  dataMat <- pivot_longer(dataMat,cols=1:length(cols)) 
  sdMat <- pivot_longer(sdMat,cols=1:length(cols))
  finalMat <- inner_join(dataMat,sdMat,by=c("name","Time"))

  colnames(finalMat) = c("Time","Method","Mean","SD")
  
    
  p <- ggplot(finalMat[finalMat$Time>timeCutoff,],aes(x=Time, y=Mean, group=Method,color=Method,fill=Method))
  p <- p + geom_ribbon(aes(x=Time,ymin = Mean-1.96*SD, ymax=Mean+1.96*SD),alpha=0.5,color=NA)
  p <- p + geom_line(size=2) 
  p <- p + xlab(xlabel) + ylab(ylabel)  
  for(i in 1:(floor(T/gaps))){
    if(i*gaps != 1 & i*gaps !=T){
        p <- p + geom_vline(xintercept = i*gaps,linetype="dashed",color="grey",size = 4)
    }
  }
  p <- p +  theme_bw() + theme(axis.text=element_text(size=25),axis.title=element_text(size=60,face="plain")) 
  return(p)
}

alphaPlots <- list()
muPlots <- list()
differencesPlots <- list()
gammaPlots <- list()
covPlots <- list()
covDiffPlots <- list()
covCumulativeDiffPlots <- list()

for(i in 1:length(mus)){
    alphaPlots[[i]] <- makePlot(alphaList[[i]],alphaSdList[[i]],T=length(mus[[i]]),gaps=gapsVec[i],
                                cols = c("DtACI","AgACI","MVP","AlphaStar"),xlabel = "Time",
                                ylabel = TeX("$\\alpha_t/\\alpha^*_t$"),colors=c(4,1,2,3),
                                legendNames = c("DtACI","AgACI","MVP",TeX(" $\\alpha^*_t$")))
    alphaPlots[[i]] <- alphaPlots[[i]] + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3,"AlphaStar"=1),
                                                             labels=c("DtACI","AgACI","MVP",TeX(" $\\alpha^*_t$")),
                                                             aesthetics = c("color", "fill")) 
    muPlots[[i]] <- makePlot(matrix(mus[[i]],ncol=1),matrix(rep(0,length(mus[[i]])),ncol=1),T=length(mus[[i]]),gaps=gapsVec[i],
                                cols = c("mu"),xlabel = "Time",
                                ylabel = TeX("$\\mu_t$"),colors=c(1),
                                legendNames = c("mu"))
    muPlots[[i]] <- muPlots[[i]] + scale_colour_manual(name="Method",values = c("mu"=1),
                                                             labels=c("mu"),
                                                             aesthetics = c("color", "fill")) 
    differencesPlots[[i]] <- makePlot(diffList[[i]],diffSdList[[i]],T=length(mus[[i]]),gaps=gapsVec[i],
                                      cols = c("DtACI","AgACI","MVP"),xlabel = "Time",
                                      ylabel = TeX("$|\\bar{\\alpha}_t - \\alpha^*_t|$"),colors=c(4,2,3))
    differencesPlots[[i]] <- differencesPlots[[i]] + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3),
                                                             labels=c("DtACI","AgACI","MVP"),
                                                             aesthetics = c("color", "fill")) 
    gammaPlots[[i]] <- makePlot(gammaList[[i]],gammaSdList[[i]],T=length(mus[[i]]),gaps=gapsVec[i],
                                cols = c("DtACI","AgACI"),xlabel = "Time",
                                ylabel = TeX("$\\bar{\\gamma}_t$"))
    gammaPlots[[i]] <- gammaPlots[[i]] + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2),
                                                             labels=c("DtACI","AgACI"),
                                                             aesthetics = c("color", "fill")) 
    covPlots[[i]] <- makePlot(covList[[i]],covSdList[[i]],T=length(mus[[i]]),gaps=gapsVec[i],
                                cols = c("DtACI","AgACI","MVP"),xlabel = "Time",
                                ylabel = "Coverage Gap",colors=c(4,2,3),
                                legendNames = c("DtACI","AgACI","MVP",TeX(" $\\alpha^*_t$")))
    covPlots[[i]] <- covPlots[[i]] + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3),
                                                             labels=c("DtACI","AgACI","MVP",TeX(" $\\alpha^*_t$")),
                                                             aesthetics = c("color", "fill")) 
    covDiffPlots[[i]] <- makePlot(covDiffList[[i]],covDiffSdList[[i]],T=length(mus[[i]]),gaps=gapsVec[i],
                                cols = c("AgACI","MVP"),xlabel = "Time",
                                ylabel = "Difference in Coverage Gap",colors=c(2,3),
                                legendNames = c("AgACI","MVP"))
    covDiffPlots[[i]] <- covDiffPlots[[i]] + scale_colour_manual(name="Method",values = c("AgACI"=2,"MVP"=3),
                                                             labels=c("AgACIvs.DtACI","MVPvs.DtACI"),
                                                             aesthetics = c("color", "fill")) 
    covDiffPlots[[i]] <- covDiffPlots[[i]] + geom_hline(yintercept = 0, linetype = "dashed", color = "black", size = 2)
    covCumulativeDiffPlots[[i]] <- makePlot(covCumulativeDiffList[[i]],covCumulativeDiffSdList[[i]],T=length(mus[[i]]),gaps=gapsVec[i],
                                cols = c("DtACI","AgACI","MVP"), xlabel = "Time",
                                ylabel = "Cumulative Difference in Coverage Gap",colors=c(4,2,3),
                                legendNames = c("GaACI","AgACI","MVP"))
    covCumulativeDiffPlots[[i]] <- covCumulativeDiffPlots[[i]] + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3),
                                                             labels=c("DtACI","AgACI","MVP"),
                                                             aesthetics = c("color", "fill")) 
}

legend1 <- get_legend(alphaPlots[[1]]+theme(legend.position = "top",legend.text=element_text(size=50),legend.title=element_blank(),
                                           legend.key.size = unit(4,"line")))
legend2 <- get_legend(covDiffPlots[[1]]+theme(legend.position = "top",legend.text=element_text(size=50),legend.title=element_blank(),
                                             legend.key.size = unit(4,"line")))
legend3 <- get_legend(gammaPlots[[1]]+theme(legend.position = "top",legend.text=element_text(size=50),legend.title=element_blank(),
                                           legend.key.size = unit(4,"line")))

blankplot <- ggplot()+geom_blank(aes(1,1)) + cowplot::theme_nothing()
for(i in 1:length(mus)){
    alphaPlots[[i]] <- alphaPlots[[i]] + theme(legend.position = "none")
    alphaPlots[[i]] <- alphaPlots[[i]] + ylim(0,0.6)
    muPlots[[i]] <- muPlots[[i]] +  theme(legend.position = "none")
    muPlots[[i]] <- muPlots[[i]] + ylim(-1.6,1.6)
    differencesPlots[[i]] <- differencesPlots[[i]] + theme(legend.position = "none")
    gammaPlots[[i]] <- gammaPlots[[i]]  +  theme(legend.position = "none") 
    gammaPlots[[i]] <- gammaPlots[[i]] + ylim(0,0.12) + theme(axis.text=element_text(size=40),axis.title=element_text(size=50,face="plain"))
    covPlots[[i]] <- covPlots[[i]]  +  theme(legend.position = "none")
    covDiffPlots[[i]] <- covDiffPlots[[i]]  +  theme(legend.position = "none")
    covDiffPlots[[i]] <- covDiffPlots[[i]] + ylim(-0.05,0.32) + theme(axis.text=element_text(size=40),axis.title=element_text(size=50,face="plain"))
    covCumulativeDiffPlots[[i]] <- covCumulativeDiffPlots[[i]] + theme(legend.position = "none")
    if(i < 3){
        #alphaPlots[[i]] <- alphaPlots[[i]] + 
        #                    theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
        differencesPlots[[i]] <- differencesPlots[[i]] + 
                            theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
        gammaPlots[[i]] <- gammaPlots[[i]] + 
                            theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
        covPlots[[i]] <- covPlots[[i]] + 
                            theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
        covDiffPlots[[i]] <- covDiffPlots[[i]] + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
        covCumulativeDiffPlots[[i]] <- covCumulativeDiffPlots[[i]] + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
    }
    if(i>1){
        alphaPlots[[i]] <- alphaPlots[[i]] + 
                            theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())
        muPlots[[i]] <- muPlots[[i]] +  theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())
    }
}

alphaPlots[[1]] <- alphaPlots[[1]] +  annotate('text',x=1000, y=0.58, label="Stationary",size=16,fontface="bold")
alphaPlots[[2]] <- alphaPlots[[2]] +  annotate('text',x=820, y=0.58, label="Smooth",size=16,fontface="bold")
alphaPlots[[3]] <- alphaPlots[[3]] +  annotate('text',x=550, y=0.58, label="Jump",size=16,fontface="bold")

muPlots[[1]] <- muPlots[[1]] +  annotate('text',x=1000, y=1.5, label="Stationary",size=16,fontface="bold")
muPlots[[2]] <- muPlots[[2]] +  annotate('text',x=820, y=1.5, label="Smooth",size=16,fontface="bold")
muPlots[[3]] <- muPlots[[3]] +  annotate('text',x=550, y=1.5, label="Jump",size=16,fontface="bold")


pdf("Figures/AlphaStarTrajectories.pdf", width=25, height = 9.5)
grid.arrange(legend1,blankplot,blankplot,
             alphaPlots[[1]],alphaPlots[[2]],alphaPlots[[3]],widths = c(4.7,3.3,3.3),heights = c(0.5, 2.5),ncol=3)
dev.off()


Attaching package: ‘dplyr’


The following object is masked from ‘package:gridExtra’:

    combine


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”
Warning message in get_plot_component(plot, "guide-box"):
“Multiple components found; returning the first one. To return all, use `return_all = TRUE`.”
Warning message in get_plot_component(plot, "guide-box"):
“Multiple components found; returning the first one. To return all, use `return_all = TRUE`.”
Warning message in get_plot_component(plot, "guide-box"):
“Multiple components found; returning the first one. To return all, use `return_all = TRUE`.”
Warning message:
“Removed 47 rows containing missing values or values outside the scale range
(`geom_line()`).”
Warning message:
“Removed 49 rows contai

pdf 
  2

In [6]:
pdf("Figures/muTrajectories.pdf", width=25, height = 9.5)
grid.arrange(muPlots[[1]],muPlots[[2]],muPlots[[3]],widths = c(4.3,3.3,3.3),ncol=3)
dev.off()

pdf 
  2

In [7]:
options(repr.plot.width=25, repr.plot.height=30)

covDiffPlots[[1]] <- covDiffPlots[[1]] +  annotate('text',x=540, y=0.31, label="Stationary",size=16,fontface="bold")
covDiffPlots[[2]] <- covDiffPlots[[2]] +  annotate('text',x=375, y=0.31, label="Smooth",size=16,fontface="bold")
covDiffPlots[[3]] <- covDiffPlots[[3]] +  annotate('text',x=160, y=0.31, label="Jump",size=16,fontface="bold")
gammaPlots[[1]] <- gammaPlots[[1]] +  annotate('text',x=590, y=0.1155, label="Stationary",size=16,fontface="bold")
gammaPlots[[2]] <- gammaPlots[[2]] +  annotate('text',x=360, y=0.1155, label="Smooth",size=16,fontface="bold")
gammaPlots[[3]] <- gammaPlots[[3]] +  annotate('text',x=200, y=0.1155, label="Jump",size=16,fontface="bold")


pdf("Figures/SimulatedDataResults.pdf", width=30, height = 35)
grid.arrange(legend2,legend3,covDiffPlots[[1]],gammaPlots[[1]],covDiffPlots[[2]],gammaPlots[[2]],
             covDiffPlots[[3]],gammaPlots[[3]],heights = c(0.2, rep(2.5,3)),ncol=2)
    
dev.off()

Warning message:
“Removed 62 rows containing missing values or values outside the scale range
(`geom_line()`).”
Warning message:
“Removed 61 rows containing missing values or values outside the scale range
(`geom_line()`).”
Warning message:
“Removed 62 rows containing missing values or values outside the scale range
(`geom_line()`).”


pdf 
  2